Instantiate DuckDB and create the tables with their relationships.

The goal of this exercise is to understand how GraphQL maps a SQL database to the GraphQL model.

In [ ]:
import duckdb
from pprint import pprint

con = duckdb.connect(database=':memory:')

con.execute("""
CREATE TABLE employee (
    employee_id INTEGER PRIMARY KEY,
    name VARCHAR,
    number INTEGER
);
""")
con.execute("""
CREATE TABLE checks (
    check_id INTEGER PRIMARY KEY,
    employee_id INTEGER,
    action VARCHAR,
    time TIMESTAMP,
    FOREIGN KEY(employee_id) REFERENCES employee(employee_id)
);
""")
con.executemany(
    "INSERT INTO employee (employee_id, name, number) VALUES (?, ?, ?);",
    [
        (1, "Alice", 101),
        (2, "Bob", 102),
        (3, "Carol", 103)
    ]
)

con.executemany(
    "INSERT INTO checks (check_id, employee_id, action, time) VALUES (?, ?, ?, ?);",
    [
        (101, 1, "enter", "2025-11-25 08:00:00"),
        (102, 1, "leave", "2025-11-25 17:00:00"),
        (103, 2, "enter", "2025-11-24 09:00:00"),
        (104, 2, "leave", "2025-11-24 18:00:00"),
        (105, 3, "enter", "2025-11-23 07:30:00"),
        (106, 3, "leave", "2025-11-23 15:30:00"),
        (107, 1, "enter", "2025-11-26 08:10:00"),
        (108, 1, "leave", "2025-11-26 13:00:00")
    ]
)



Once the data types are defined in the database, they must also be defined as Python classes.
* 1.- Map the data type and its properties.
* 2.- Create a class derived from `InputObjectType` to insert it directly.

The important thing is that `Input` and `ObjectType` must have the same fields.

In [95]:
import graphene

class Employee(graphene.ObjectType):
    employee_id = graphene.Int()
    name        = graphene.String()
    number      = graphene.Int()

class Check(graphene.ObjectType):
    check_id    = graphene.Int()
    employee_id = graphene.Int()
    action      = graphene.String()
    time        = graphene.String()

# Se utilizan para insertar directo
class EmployeeInput(graphene.InputObjectType):
    employee_id = graphene.Int(required=True)
    name        = graphene.String(required=True)
    number      = graphene.Int(required=True)

class CheckInput(graphene.InputObjectType):
    check_id    = graphene.Int(required=True)
    employee_id = graphene.Int(required=True)
    action      = graphene.String(required=True)
    time        = graphene.String(required=True)

pprint(con.execute("SELECT * FROM checks;").fetchall())


[(101, 1, 'enter', datetime.datetime(2025, 11, 25, 8, 0)),
 (102, 1, 'leave', datetime.datetime(2025, 11, 25, 17, 0)),
 (103, 2, 'enter', datetime.datetime(2025, 11, 24, 9, 0)),
 (104, 2, 'leave', datetime.datetime(2025, 11, 24, 18, 0)),
 (105, 3, 'enter', datetime.datetime(2025, 11, 23, 7, 30)),
 (106, 3, 'leave', datetime.datetime(2025, 11, 23, 15, 30)),
 (107, 1, 'enter', datetime.datetime(2025, 11, 26, 8, 10)),
 (108, 1, 'leave', datetime.datetime(2025, 11, 26, 13, 0))]


You must define a `QUERY` class. The responsibilities of this class are as follows:
* It serves as a reading interface for the data. This data can come from any data source.
* The methods named `resolve_` + `variable` define how the data will be provided.
* It can handle exceptions in a manner similar to a getter or setter.

In [96]:

class Query(graphene.ObjectType):
    # Estas variables se acceden a travez deL query
    # limit y employe_id se tienen que definir aqui, para poder ser usadas en el esquema
    employees = graphene.List(Employee, limit=graphene.Int())
    checks    = graphene.List(Check, employee_id=graphene.Int())

    # En este ejemplo introducimos una regla de negocio donde no dar un limite equivale a dar 
    # toda la lista de empleados
    def resolve_employees(root, info, limit=None):
        q, p = (
            ("SELECT employee_id, name, number FROM employee LIMIT ?;", [limit])
            if limit is not None
            else ("SELECT employee_id, name, number FROM employee;", [])
        )
        result = con.execute(q, p).fetchall()
        return [Employee(employee_id=r[0], name=r[1], number=r[2]) for r in result]

    def resolve_checks(root, info, employee_id=None):
        try:
            result = con.execute(
                "SELECT check_id, employee_id, action, time FROM checks WHERE employee_id = ?;",
                [employee_id]
            ).fetchall()
            return [
                Check(
                    check_id    = r[0],
                    employee_id = r[1],
                    action      = r[2],
                    time        = str(r[3])
                ) for r in result
            ]
        except Exception as e:
            return []

Mutations: It represents the implementation of common CRUD operations. It is common to have
certain flags that represents errors. For example the DB might throw FK exceptions or 
duplicate keys which are returned as success or not.
* 1.- The Arguments is an InputType which was defined before. 
* 2.- The employee property tells graphql that can be selected for specific fields.
* 3.- EmployeeInput is useful to simplify the insertion query.

In [97]:
class CreateEmployee(graphene.Mutation):
    class Arguments:
        employee_data = EmployeeInput(required=True)

    connection_ok = graphene.Boolean()
    constraint_ok = graphene.Boolean()
    employee = graphene.Field(lambda: Employee)

    def mutate(root, info, employee_data):
        connection_ok = True
        constraint_ok = True
        try:
            con.execute(
                "INSERT INTO employee (employee_id, name, number) VALUES (?, ?, ?);",
                [employee_data.employee_id, employee_data.name, employee_data.number]
            )
        except duckdb.ConstraintException as c_exc:
            # print(f"Constraint error: {c_exc}")
            constraint_ok = False
        except duckdb.ConnectionException as conn_exc:
            # print(f"Connection error: {conn_exc}")
            connection_ok = False
        except Exception as e:
            print(f"Other error: {e}")
            connection_ok = False
            constraint_ok = False

        employee = Employee(
            employee_id=employee_data.employee_id,
            name=employee_data.name,
            number=employee_data.number
        )
        return CreateEmployee(employee=employee, connection_ok=connection_ok, constraint_ok=constraint_ok)

class CreateCheck(graphene.Mutation):
    class Arguments:
        check_data = CheckInput(required=True)

    connection_ok = graphene.Boolean()
    constraint_ok = graphene.Boolean()
    is_key_unique = graphene.Boolean()

    check = graphene.Field(lambda: Check)

    def mutate(root, info, check_data):
        connection_ok = True
        constraint_ok = True
        is_key_unique = True
        try:
            con.execute(
                "INSERT INTO checks (check_id, employee_id, action, time) VALUES (?, ?, ?, ?);",
                [check_data.check_id, check_data.employee_id, check_data.action, check_data.time]
            )
        except duckdb.ConstraintException as c_exc:
            msg = str(c_exc).lower()
            if 'duplicate' in msg or 'unique' in msg or 'primary key' in msg:
                is_key_unique = False
            if 'foreign key' in msg:
                constraint_ok = False
        except duckdb.ConnectionException as conn_exc:
            connection_ok = False
        except Exception as e:
            print(f"Other error: {e}")
            connection_ok = False
            constraint_ok = False
            is_key_unique = False
        # Return the attempted check that was or not inserted
        check = Check(
            check_id    = check_data.check_id,
            employee_id = check_data.employee_id,
            action      = check_data.action,
            time        = check_data.time
        )
        return CreateCheck(check=check, 
                           connection_ok=connection_ok, 
                           constraint_ok=constraint_ok,
                           is_key_unique=is_key_unique
                           )

In [98]:
# Registrar operaciones CRUD
class Mutations(graphene.ObjectType):
    create_employee = CreateEmployee.Field()
    create_check = CreateCheck.Field()

schema = graphene.Schema(query=Query, mutation=Mutations)

With these schemas built we can model an insertion into DB.
* 1.- The first run will succeed as the key exists and is not repeated
* 2.- The second run will fail as the primary_key already exists.

In [100]:
from pprint import pprint
graph_request = '''
    mutation {
        createCheck( checkData: {
            checkId: 999
            employeeId: 2
            action: "entrada"
            time: "2025-11-27T12:00:00"
        }) 
        {
            
            employeeExists:constraintOk
            checkIdIsUnique:isKeyUnique
            check 
            {
                
                id_accion:checkId
                id_empleado:employeeId
            }
        }
    }
'''
res = schema.execute(graph_request)
pprint(res.data)
pprint(res.errors)


{'createCheck': {'check': {'id_accion': 999, 'id_empleado': 2},
                 'checkIdIsUnique': False,
                 'employeeExists': True}}
None
